#  Deep Learning
Assignment 2¶
Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.
The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb

In [6]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile 
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range 

Reload the data we genereted in the pickle file 

In [7]:
pickle_file='notMNIST.pickle'


with open(pickle_file , 'rb') as f:
    save=pickle.load(f)
    train_dataset=save ['train_dataset']
    train_labels=save ['train_labels']
    valid_dataset=save ['valid_dataset']
    valid_labels=save ['valid_labels']
    test_dataset=save ['test_dataset']
    test_labels=save ['test_labels']
    del save #to help compiler to free up space 
    print('train_dataset', train_dataset.shape , train_labels.shape)
    print('validation set ', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape, test_labels.shape)

train_dataset (200000, 28, 28) (200000,)
validation set  (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


#  reformat into a shape that is more adapted to the models we are going to train
DATA AS FLAT MATRIX
LAB ELS AS FLOAT 1-HOT ENCODINGS

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


# Now firstly train a multinomial logistic regression (softmax regression)using simple gradient descent
how tensorflow gonna work in here :
firstly give all the inputs variable and the formula of our desired computition then these created as nodes over a computation graph 
with graph.as_default():



In [11]:
#with gradient descent training we have use small datasets as it takes longer  time
#So we should subset the train data for faster training 

train_subset=10000
graph=tf.Graph()
with graph.as_default():
    
    #Input data.
    #Load the training ,validation and data into constants that are attached to the graph
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset= tf.constant(test_dataset)
    
    #variables
    #these are the parameters that we are going to trainng .the weight matrix in inisialized using random values 
    #with normal distribution
    #the biases get initialized to zero
    
    weights=tf.Variable(
     tf.truncated_normal([image_size*image_size, num_labels]))
    #print('weights', weights)
    biases =tf.Variable(tf.zeros([num_labels]))
    #print ('biases', biases)
    
    
    #training computation
    #multiply the inputs with the weight matrix , and add biases then compute softmax and cross-entropy 
    #then take the average of this cross-entropy across all training example : that is our loss
    
    logits = tf.matmul(tf_train_dataset, weights)+ biases #scores
    loss=  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))#cross-entropy
    
    #optimizer
    #now using gradient descent for minimizing the loss after cross entropy
    
    optimizer= tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #predictions for the training , validation and test data
    #these are not part of training , but merely ,here so that we can report
    #accuracy figure as we train.
    
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf_valid_dataset,weights) + biases)
    test_prediction= tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+ biases)
    
def accuracy(predictions, labels):
    return(100.0 * np.sum(np.argmax(predictions, 1)== np.argmax(labels, 1))/ predictions.shape[0])
    
    
    

now run the computation graph iteratly 

In [61]:
num_steps= 801




with tf.Session(graph=graph) as session :
    # this is a one-time operation which ensures the parameters initialized as we described in the graph: random
    #weights for matrix and zeros for the bias 
    
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        #run the computations ,we tell . run() that we want to run the optimizer,
        #and get the loss value and the training predictions returnes as numpy arrays
        _, l, predictions = session.run([optimizer , loss , train_prediction])
        if (step % 100 == 0):
            print('loss at step %d: %f' % (step, l))
            print('training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, : ]))
            #Calling.evaL() on valid_prediction is basically like calling run (), but just to get that one np array 
            #and it recomputes all its graph dependencies
            print('validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
session.close()
         
        
        

initialized
loss at step 0: 18.780605
training accuracy: 10.9%
validation accuracy: 14.1%
loss at step 100: 2.165911
training accuracy: 71.8%
validation accuracy: 71.5%
loss at step 200: 1.748098
training accuracy: 74.5%
validation accuracy: 73.5%
loss at step 300: 1.526161
training accuracy: 75.9%
validation accuracy: 74.3%
loss at step 400: 1.377679
training accuracy: 76.7%
validation accuracy: 74.8%
loss at step 500: 1.268389
training accuracy: 77.3%
validation accuracy: 75.1%
loss at step 600: 1.182925
training accuracy: 77.9%
validation accuracy: 75.4%
loss at step 700: 1.113233
training accuracy: 78.4%
validation accuracy: 75.5%
loss at step 800: 1.054808
training accuracy: 78.7%
validation accuracy: 75.5%
test accuracy: 83.0%


# Now we are going to use Stochastic Gradient decent (SDG)
This is much faster.
The graph will be similar but this time we will not use a constant node to store the training data.
we will create a placeholder node which will be fed actual data at every call of session.run()

In [12]:
batch_size=128
graph=tf.Graph()
with graph.as_default():
    #Input data. For the training data, we use a placeholder that will be fed 
    #at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights= tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))##randomly selected value
    biases = tf.Variable(tf.zeros([num_labels]))##biases =0
    
    # Training Computation.
    logits =tf.matmul(tf_train_dataset,weights)+biases ##y(logits)=wx+b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights)+ biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)

Let's run with SDG:

In [14]:
num_steps=30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _, l,predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        '''
        if (step % 500==0):
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
        print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(),test_labels))
        '''
        print(accuracy(test_prediction.eval(),test_labels))
session.close()

Initialized
11.35
15.93
21.47
29.75
34.7
39.0
43.6
46.86
49.84
51.47
54.12
54.52
57.92
58.84
60.11
60.64
61.34
61.68
62.09
62.98
63.45
63.54
64.49
65.66
66.15
67.66
68.26
68.41
68.01
68.34
68.82
68.56
69.5
69.54
70.28
70.82
70.38
70.67
71.31
71.38
71.47
72.16
72.56
72.37
72.85
72.76
73.47
74.07
74.13
73.86
74.01
74.04
74.65
74.66
74.89
74.67
75.27
74.68
74.75
75.18
75.24
75.61
75.57
75.73
75.77
75.72
75.76
75.28
75.74
76.36
76.54
76.04
76.62
76.43
76.84
76.73
76.58
76.42
77.1
77.68
77.37
77.75
77.46
77.7
77.92
75.56
77.35
77.66
77.62
77.85
78.11
78.21
78.34
78.12
77.72
77.62
78.1
78.74
78.9
78.73
78.71
78.79
78.56
78.49
78.68
78.54
78.37
78.28
78.24
78.6
78.97
78.84
78.69
78.91
79.04
79.26
79.3
79.39
79.53
79.4
79.15
79.62
79.9
79.72
79.84
80.02
80.2
80.15
80.41
80.22
79.95
79.71
80.18
80.26
80.71
80.6
80.4
80.58
80.83
80.48
79.73
80.1
80.08
80.21
80.07
80.38
80.51
80.76
80.8
80.89
80.37
78.36
80.02
80.32
80.46
80.37
80.41
80.88
80.71
80.97
80.74
81.05
81.12
81.22
80.82
80.5
80.73
80.9

In [64]:
batch_size=128
num_nodes=1024
#SGDSTEP=0.5
#NumberofLayers=2 #(front layer+1 hidden layer)

'''
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial:tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
        '''
    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
    biases_1=tf.Variable(tf.zeros([num_nodes]))
    weights_2=tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2=tf.Variable(tf.zeros(num_labels))
    
    #training Computation.
    relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+ biases_1)
    logits=tf.matmul(relu_layer,weights_2)+biases_2
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)+biases_2)
    test_predicton= tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)
    
    
   
    print(graph)
      
    

In [65]:
num_steps=30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _, l,predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500==0):
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
        print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(),test_labels))
session.close()

Initialized
Minibatch loss at step 0: 316.972229
Minibatch accuracy: 39.2%


ValueError: Cannot use the default session to evaluate tensor: the tensor's graph is different from the session's graph. Pass an explicit session to `eval(session=sess)`.